# CityBikes

Send a request to CityBikes for the city of your choice. 

In [3]:
import pandas as pd
import requests
# 
#url = "http://api.citybik.es/v2/networks/velobike-moscow" /API endpoint URL (velobike-moscow) - didn't work for me
url="http://api.citybik.es/v2/networks/bixi-toronto" # /API endpoint URL (bixi-toronto)
response = requests.get(url)
if response.status_code == 200: # Check if the request was successful
    data_dict = response.json() # Parse JSON data into a Python dictionary
    stations = data_dict["network"]["stations"] # Extract the list of stations from the dictionary
    df = pd.DataFrame(stations) # Create a DataFrame
else:
    print("Failed to fetch data from the API.")


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [4]:
bike_stations=[] # creating empty bike_stations list where will collect requested info
for station in stations: # starting for loop to go through all rows in stations by filtering requested fields 
    station_id=station['id']
    name=station['name']
    latitude=station['latitude']
    longitude=station['longitude']
    number_of_bikes=station['free_bikes']
    # append returned values to bike_station list. 
    bike_stations.append({"Station_id":station_id,
                          "Name":name,
                          "Latitude":latitude,
                          "Longitude":longitude,
                          "Free Bikes": number_of_bikes
                          
                         })

Put your parsed results into a DataFrame.

In [5]:
# create a dafaframe from bike_stations list
df_bike=pd.DataFrame(bike_stations)
# Store the DataFrame in the IPython store
%store df_bike
#df.to_csv('city_bikes_ll_2.csv',index=False) // was using/testing "csv" file as an alterantive source for data

Stored 'df_bike' (DataFrame)


In [6]:
df_bike # testing dataframe

,Station_id,Name,Latitude,Longitude,Free Bikes
0,fb337bbed72e2be090071e199899b2be,Queen St E / Woodward Ave,43.665269,-79.319796,18
1,4ff88d5880e71aa40d34cfe5d09b0ca7,Primrose Ave / Davenport Rd,43.671420,-79.445947,2
2,a09c67c0b419654d907c9134b108e328,Queen St. E / Rhodes Ave.,43.666224,-79.317693,21
3,d6a9daee68070a8b106cfb598d81308c,Bond St / Queen St E,43.653236,-79.376716,3
4,8f8af40d9388c8a3962559e8681d3db7,Church St / Alexander St,43.663722,-79.380288,13
...,...,...,...,...,...
676,44d245c021bcfb69fed5e4b088d60dd5,Eastgate Ct / Pharmacy Ave,43.711044,-79.291273,7
677,84e8e3827807916c9c3ac9b60404346d,Firvalley Ct / Warden Ave,43.703211,-79.278715,11
678,84dc7d9185e88915466d12f23fe07753,Midland Ave/ Kingston Rd,43.712858,-79.248709,1
679,6228f42d28032bf18c0a7f9411df5fa0,Bellamy Rd S / Kingston Rd,43.732328,-79.224966,0


In [11]:
# data audit and cleaning (same process can be used for yelp and frq)

# Using median to fill missing values
median_free_bikes = df_bike['Free Bikes'].median()
updated_values_count = df_bike['Free Bikes'].fillna(median_free_bikes, inplace=True)
print(f"Number of missing values updated with the median: {updated_values_count}")

# Removing Duplicates
duplicates_before = len(df_bike)
df_bike.drop_duplicates(inplace=True)
duplicates_after = len(df_bike)
print ("Removed duplicates:" ,(duplicates_before - duplicates_after))

# Converting 'Free Bikes' data type to integer
old_data_type = df_bike['Free Bikes'].dtype
df_bike['Free Bikes'] = df_bike['Free Bikes'].astype(int)
new_data_type = df_bike['Free Bikes'].dtype
updates_count = "converted" if old_data_type != new_data_type else "None converted"
print(updates_count)

# Cleaning extra spaces in 'Name' column
original_names = df_bike['Name']
df_bike['Name'] = df_bike['Name'].str.strip()
changes_count = (original_names != df_bike['Name']).sum()
print(f"Number of changes made during string cleaning: {changes_count}")


Number of missing values updated with the median: None
Removed duplicates: 0
None converted
Number of changes made during string cleaning: 0
